# **EDA on Stories POS Reports**

This notebook explores the cleaned Stories datasets and provides end-to-end visual analysis for sales, profit, categories, groups, branches, and monthly trends.

## **Load Dataset**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

try:
    import plotly.express as px
    HAS_PLOTLY = True
except Exception:
    HAS_PLOTLY = False
    print('Plotly is not installed. Interactive charts will be skipped.')

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_theme(style='whitegrid', palette='Set2')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 180)


In [ ]:
BASE_PATH = Path('/content/drive/MyDrive/cleaned')
if not BASE_PATH.exists():
    if Path('./cleaned').exists():
        BASE_PATH = Path('./cleaned')
    else:
        BASE_PATH = Path('./Archive/Stories_data/cleaned')

print(f'Using data folder: {BASE_PATH.resolve()}')

paths = {
    'item_profit': BASE_PATH / 'rep_00014_theoretical_profit_by_item_clean.csv',
    'group_sales': BASE_PATH / 'rep_00191_sales_by_items_by_group_clean.csv',
    'category_profit': BASE_PATH / 'rep_00673_theoretical_profit_by_category_clean.csv',
    'monthly_sales_wide': BASE_PATH / 'rep_00134_comparative_monthly_sales_clean_wide.csv',
    'monthly_sales_long': BASE_PATH / 'rep_00134_comparative_monthly_sales_clean_long.csv'
}

for key, p in paths.items():
    print(f'{key:20s} -> {p} | exists={p.exists()}')


In [ ]:
df_item = pd.read_csv(paths['item_profit'])
df_group = pd.read_csv(paths['group_sales'])
df_cat = pd.read_csv(paths['category_profit'])
df_month_wide = pd.read_csv(paths['monthly_sales_wide'])
df_month_long = pd.read_csv(paths['monthly_sales_long'])

datasets = {
    'item_profit': df_item,
    'group_sales': df_group,
    'category_profit': df_cat,
    'monthly_sales_wide': df_month_wide,
    'monthly_sales_long': df_month_long
}

for name, df in datasets.items():
    print(f'{name:20s} shape={df.shape}')


In [ ]:
# Ensure numeric columns are numeric
item_num_cols = ['qty', 'total_price', 'total_cost', 'total_cost_pct', 'total_profit', 'total_profit_pct']
group_num_cols = ['qty', 'total_amount']
cat_num_cols = ['qty', 'total_price', 'total_cost', 'total_cost_pct', 'total_profit', 'total_profit_pct']
month_wide_num_cols = ['year', 'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december', 'total_by_year']
month_long_num_cols = ['year', 'month_number', 'sales_amount']

for col in item_num_cols:
    df_item[col] = pd.to_numeric(df_item[col], errors='coerce')
for col in group_num_cols:
    df_group[col] = pd.to_numeric(df_group[col], errors='coerce')
for col in cat_num_cols:
    df_cat[col] = pd.to_numeric(df_cat[col], errors='coerce')
for col in month_wide_num_cols:
    if col in df_month_wide.columns:
        df_month_wide[col] = pd.to_numeric(df_month_wide[col], errors='coerce')
for col in month_long_num_cols:
    if col in df_month_long.columns:
        df_month_long[col] = pd.to_numeric(df_month_long[col], errors='coerce')


In [ ]:
def quick_profile(name, df, head_n=3):
    print(f'\n{name} | rows={len(df):,} cols={df.shape[1]}')
    display(df.head(head_n))
    display(df.dtypes.rename('dtype').to_frame())

for name, df in datasets.items():
    quick_profile(name, df)


## **Distribution Analysis**

### **Data Size**

In [ ]:
size_df = pd.DataFrame({
    'dataset': list(datasets.keys()),
    'rows': [len(df) for df in datasets.values()],
    'columns': [df.shape[1] for df in datasets.values()]
})

display(size_df)

fig, ax = plt.subplots(1, 2, figsize=(14, 4))
sns.barplot(data=size_df, x='dataset', y='rows', ax=ax[0], color='#4C78A8')
ax[0].set_title('Rows per Dataset')
ax[0].tick_params(axis='x', rotation=25)

sns.barplot(data=size_df, x='dataset', y='columns', ax=ax[1], color='#F58518')
ax[1].set_title('Columns per Dataset')
ax[1].tick_params(axis='x', rotation=25)

plt.tight_layout()
plt.show()


### **Missing Values**

In [ ]:
missing_tables = []
for name, df in datasets.items():
    miss = df.isna().sum().sort_values(ascending=False)
    miss = miss[miss > 0]
    if miss.empty:
        continue
    temp = pd.DataFrame({
        'dataset': name,
        'column': miss.index,
        'missing_count': miss.values,
        'missing_pct': (miss.values / len(df) * 100).round(2)
    })
    missing_tables.append(temp)

if missing_tables:
    missing_summary = pd.concat(missing_tables, ignore_index=True)
    display(missing_summary)

    pivot = missing_summary.pivot(index='dataset', columns='column', values='missing_pct').fillna(0)
    plt.figure(figsize=(14, 4))
    sns.heatmap(pivot, annot=True, fmt='.1f', cmap='YlOrRd')
    plt.title('Missing Percentage Heatmap by Dataset/Column')
    plt.tight_layout()
    plt.show()
else:
    print('No missing values detected.')


### **Summary Statistics**

In [ ]:
def numeric_summary(name, df):
    num_df = df.select_dtypes(include='number')
    if num_df.empty:
        return
    print(f'\n{name} numeric summary')
    display(num_df.describe().T)

for name, df in datasets.items():
    numeric_summary(name, df)


### **Skewness**

In [ ]:
skew_rows = []
for name, df in datasets.items():
    num_df = df.select_dtypes(include='number')
    if num_df.empty:
        continue
    sk = num_df.skew(numeric_only=True).dropna()
    for col, val in sk.items():
        skew_rows.append({'dataset': name, 'feature': col, 'skewness': val})

skew_df = pd.DataFrame(skew_rows)
display(skew_df.sort_values('skewness', key=np.abs, ascending=False).head(20))

plt.figure(figsize=(12, 5))
top_skew = skew_df.sort_values('skewness', key=np.abs, ascending=False).head(15)
sns.barplot(data=top_skew, x='skewness', y='feature', hue='dataset', dodge=False)
plt.title('Top 15 Most Skewed Numeric Features')
plt.tight_layout()
plt.show()


### **Outlier Detection**

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(16, 8))
axes = axes.flatten()

sns.boxplot(data=df_item[df_item['row_type']=='item'], x='total_price', ax=axes[0], color='#4C78A8')
axes[0].set_title('Item Total Price')

sns.boxplot(data=df_item[df_item['row_type']=='item'], x='total_profit', ax=axes[1], color='#F58518')
axes[1].set_title('Item Total Profit')

sns.boxplot(data=df_group[df_group['row_type']=='item'], x='total_amount', ax=axes[2], color='#54A24B')
axes[2].set_title('Group Sales Total Amount')

sns.boxplot(data=df_cat[df_cat['row_type']=='category'], x='total_price', ax=axes[3], color='#E45756')
axes[3].set_title('Category Total Price')

sns.boxplot(data=df_cat[df_cat['row_type']=='category'], x='total_profit', ax=axes[4], color='#72B7B2')
axes[4].set_title('Category Total Profit')

sns.boxplot(data=df_month_long[df_month_long['period_type']=='month'], x='sales_amount', ax=axes[5], color='#B279A2')
axes[5].set_title('Monthly Sales Amount')

plt.tight_layout()
plt.show()


## **Visualization Analysis**

### **Distribution of Features**

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

sns.histplot(df_item[df_item['row_type']=='item']['total_profit'].dropna(), bins=60, kde=True, ax=axes[0, 0], color='#4C78A8')
axes[0, 0].set_title('Distribution: Item Total Profit')

sns.histplot(df_group[df_group['row_type']=='item']['total_amount'].dropna(), bins=60, kde=True, ax=axes[0, 1], color='#F58518')
axes[0, 1].set_title('Distribution: Item Total Amount (Group Report)')

sns.histplot(df_cat[df_cat['row_type']=='category']['total_profit'].dropna(), bins=30, kde=True, ax=axes[1, 0], color='#54A24B')
axes[1, 0].set_title('Distribution: Category Total Profit')

sns.histplot(df_month_long[df_month_long['period_type']=='month']['sales_amount'].dropna(), bins=40, kde=True, ax=axes[1, 1], color='#E45756')
axes[1, 1].set_title('Distribution: Monthly Sales Amount')

plt.tight_layout()
plt.show()


In [ ]:
# Top and bottom products by profitability
item_only = df_item[df_item['row_type']=='item'].copy()

best_products = item_only.groupby('product_desc', as_index=False)['total_profit'].sum().nlargest(15, 'total_profit')
worst_products = item_only.groupby('product_desc', as_index=False)['total_profit'].sum().nsmallest(15, 'total_profit')

fig, axes = plt.subplots(1, 2, figsize=(18, 6))

sns.barplot(data=best_products, x='total_profit', y='product_desc', ax=axes[0], color='#54A24B')
axes[0].set_title('Top 15 Products by Total Profit')

sns.barplot(data=worst_products, x='total_profit', y='product_desc', ax=axes[1], color='#E45756')
axes[1].set_title('Bottom 15 Products by Total Profit (Loss Makers)')

plt.tight_layout()
plt.show()


In [ ]:
# Top groups by sales amount
group_totals = df_group[df_group['row_type']=='group_total'].copy()

top_groups = group_totals.groupby('group', as_index=False)['total_amount'].sum().nlargest(20, 'total_amount')

plt.figure(figsize=(10, 8))
sns.barplot(data=top_groups, x='total_amount', y='group', color='#4C78A8')
plt.title('Top 20 Groups by Total Amount')
plt.tight_layout()
plt.show()


In [ ]:
# Branch-level totals from three reports
item_branch = df_item[df_item['row_type']=='branch_total'][['branch', 'total_price', 'total_profit']].rename(columns={'total_price': 'item_total_price', 'total_profit': 'item_total_profit'})
group_branch = df_group[df_group['row_type']=='branch_total'][['branch', 'total_amount']].rename(columns={'total_amount': 'group_total_amount'})
cat_branch = df_cat[df_cat['row_type']=='branch_total'][['branch', 'total_price', 'total_profit']].rename(columns={'total_price': 'cat_total_price', 'total_profit': 'cat_total_profit'})

branch_compare = item_branch.merge(group_branch, on='branch', how='outer').merge(cat_branch, on='branch', how='outer')
display(branch_compare.head())

plot_df = branch_compare.sort_values('group_total_amount', ascending=False).head(15)
plt.figure(figsize=(12, 6))
sns.barplot(data=plot_df, x='group_total_amount', y='branch', color='#F58518')
plt.title('Top 15 Branches by Group Sales Total Amount')
plt.tight_layout()
plt.show()


### **Feature Interactions**

In [ ]:
sample_item = df_item[df_item['row_type']=='item'].dropna(subset=['total_cost', 'total_profit']).sample(min(3000, len(df_item[df_item['row_type']=='item'])), random_state=42)

plt.figure(figsize=(8, 6))
sns.scatterplot(data=sample_item, x='total_cost', y='total_profit', alpha=0.35, s=20)
plt.title('Item-Level Cost vs Profit (sample)')
plt.tight_layout()
plt.show()


In [ ]:
num_cols_item = ['qty', 'total_price', 'total_cost', 'total_cost_pct', 'total_profit', 'total_profit_pct']
item_corr = df_item[df_item['row_type']=='item'][num_cols_item].corr(numeric_only=True)

plt.figure(figsize=(8, 6))
sns.heatmap(item_corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0)
plt.title('Correlation Heatmap: Item Profit Dataset')
plt.tight_layout()
plt.show()


In [ ]:
num_cols_cat = ['qty', 'total_price', 'total_cost', 'total_cost_pct', 'total_profit', 'total_profit_pct']
cat_corr = df_cat[df_cat['row_type']=='category'][num_cols_cat].corr(numeric_only=True)

plt.figure(figsize=(8, 6))
sns.heatmap(cat_corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0)
plt.title('Correlation Heatmap: Category Profit Dataset')
plt.tight_layout()
plt.show()


### **Relationships between features and business outputs**

In [ ]:
# Branch profitability and margin
cat_branch_totals = df_cat[df_cat['row_type']=='branch_total'].copy()
cat_branch_totals['profit_margin_pct'] = (cat_branch_totals['total_profit'] / cat_branch_totals['total_price']) * 100
cat_branch_totals = cat_branch_totals.replace([np.inf, -np.inf], np.nan)

display(cat_branch_totals[['branch', 'total_price', 'total_profit', 'profit_margin_pct']].sort_values('total_profit', ascending=False).head(10))

plt.figure(figsize=(12, 6))
sns.barplot(data=cat_branch_totals.sort_values('profit_margin_pct', ascending=False), x='profit_margin_pct', y='branch', color='#54A24B')
plt.title('Branch Profit Margin % (Category Report Branch Totals)')
plt.tight_layout()
plt.show()


In [ ]:
# Beverage vs food share per branch
cat_detail = df_cat[df_cat['row_type']=='category'].copy()
pivot_cat = cat_detail.pivot_table(index='branch', columns='category', values='total_profit', aggfunc='sum').fillna(0)

if {'BEVERAGES', 'FOOD'}.issubset(pivot_cat.columns):
    mix = pivot_cat[['BEVERAGES', 'FOOD']].copy()
    mix_pct = mix.div(mix.sum(axis=1), axis=0) * 100
    mix_pct = mix_pct.sort_values('BEVERAGES', ascending=False)

    ax = mix_pct.plot(kind='bar', stacked=True, figsize=(14, 6), color=['#4C78A8', '#F58518'])
    ax.set_ylabel('Share of Total Profit (%)')
    ax.set_title('Profit Mix by Branch: Beverages vs Food')
    plt.legend(title='Category', bbox_to_anchor=(1.02, 1), loc='upper left')
    plt.tight_layout()
    plt.show()
else:
    print('Expected BEVERAGES and FOOD categories not found.')


In [ ]:
# Monthly trend by year using long table
month_order = ['january','february','march','april','may','june','july','august','september','october','november','december']

month_df = df_month_long[(df_month_long['row_type']=='branch') & (df_month_long['period_type']=='month')].copy()
month_df['period'] = pd.Categorical(month_df['period'], categories=month_order, ordered=True)

year_month_totals = month_df.groupby(['year', 'period'], as_index=False)['sales_amount'].sum().sort_values(['year','period'])

plt.figure(figsize=(12, 5))
sns.lineplot(data=year_month_totals, x='period', y='sales_amount', hue='year', marker='o', linewidth=2)
plt.title('Monthly Sales Trend by Year (All Branches)')
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()


In [ ]:
# Top branches monthly trend (based on total annual sales)
branch_totals = month_df.groupby('branch', as_index=False)['sales_amount'].sum().nlargest(8, 'sales_amount')
top_branches = branch_totals['branch'].tolist()

top_month_df = month_df[month_df['branch'].isin(top_branches)].copy()
top_month_df = top_month_df.groupby(['branch', 'period'], as_index=False)['sales_amount'].sum()

g = sns.relplot(
    data=top_month_df,
    x='period', y='sales_amount',
    col='branch', col_wrap=4,
    kind='line', marker='o',
    height=3, aspect=1.2,
    facet_kws={'sharey': False}
)
g.set_xticklabels(rotation=45)
g.fig.subplots_adjust(top=0.90)
g.fig.suptitle('Monthly Sales Trend for Top 8 Branches')
plt.show()


In [ ]:
# Branch x month heatmap for 2025
heat_2025 = month_df[month_df['year']==2025].pivot_table(index='branch', columns='period', values='sales_amount', aggfunc='sum').reindex(columns=month_order)

plt.figure(figsize=(14, 8))
sns.heatmap(heat_2025, cmap='YlGnBu')
plt.title('2025 Sales Heatmap: Branch vs Month')
plt.tight_layout()
plt.show()


In [ ]:
# 2025 vs 2026 January comparison at branch level
jan = month_df[month_df['period']=='january'].copy()
jan_pivot = jan.pivot_table(index='branch', columns='year', values='sales_amount', aggfunc='sum').fillna(0)
if 2025 in jan_pivot.columns and 2026 in jan_pivot.columns:
    jan_pivot['change_pct'] = np.where(jan_pivot[2025]==0, np.nan, (jan_pivot[2026]-jan_pivot[2025]) / jan_pivot[2025] * 100)
    jan_cmp = jan_pivot.reset_index().sort_values('change_pct', ascending=False)
    display(jan_cmp.head(15))

    plt.figure(figsize=(12, 6))
    sns.barplot(data=jan_cmp.dropna(subset=['change_pct']).head(20), x='change_pct', y='branch', color='#72B7B2')
    plt.title('Top 20 Branches by January YoY Change % (2026 vs 2025)')
    plt.tight_layout()
    plt.show()
else:
    print('January comparison needs both 2025 and 2026 columns.')


## **Cross-Report Consistency Checks**

In [ ]:
# Compare branch total_price from item report vs category report
item_total = df_item[df_item['row_type']=='branch_total'][['branch', 'total_price']].rename(columns={'total_price': 'item_branch_total_price'})
cat_total = df_cat[df_cat['row_type']=='branch_total'][['branch', 'total_price']].rename(columns={'total_price': 'cat_branch_total_price'})

consistency = item_total.merge(cat_total, on='branch', how='inner')
consistency['abs_diff'] = (consistency['item_branch_total_price'] - consistency['cat_branch_total_price']).abs()
consistency['pct_diff'] = np.where(consistency['item_branch_total_price']==0, np.nan, consistency['abs_diff']/consistency['item_branch_total_price']*100)

consistency = consistency.sort_values('abs_diff', ascending=False)
display(consistency.head(15))

plt.figure(figsize=(8, 6))
sns.scatterplot(data=consistency, x='item_branch_total_price', y='cat_branch_total_price', s=80)
max_val = np.nanmax([consistency['item_branch_total_price'].max(), consistency['cat_branch_total_price'].max()])
plt.plot([0, max_val], [0, max_val], '--', color='gray')
plt.title('Branch Total Price: Item Report vs Category Report')
plt.tight_layout()
plt.show()


## **Interactive Visualizations (Plotly)**

In [ ]:
# Interactive: top branches by annual sales
if HAS_PLOTLY:
    branch_year_sales = month_df.groupby(['year', 'branch'], as_index=False)['sales_amount'].sum()
    fig = px.bar(
        branch_year_sales.sort_values('sales_amount', ascending=False),
        x='branch', y='sales_amount', color='year',
        title='Branch Annual Sales by Year',
        barmode='group',
        height=500
    )
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()
else:
    print('Skipping interactive chart: Plotly not available.')


In [ ]:
# Interactive: item profit vs cost bubble chart
if HAS_PLOTLY:
    bubble_df = df_item[df_item['row_type']=='item'].dropna(subset=['total_cost', 'total_profit', 'qty']).copy()
    bubble_df = bubble_df.nlargest(2500, 'total_price')

    fig = px.scatter(
        bubble_df,
        x='total_cost', y='total_profit',
        size='qty', color='branch',
        hover_data=['product_desc', 'category', 'division'],
        title='Item Cost vs Profit (bubble size = qty)',
        height=600
    )
    fig.show()
else:
    print('Skipping interactive chart: Plotly not available.')


## **Key Insights (Auto-generated)**

In [ ]:
insights = []

# 1) Top branch by sales
branch_sales = month_df.groupby('branch', as_index=False)['sales_amount'].sum().sort_values('sales_amount', ascending=False)
if not branch_sales.empty:
    insights.append(f"Top branch by monthly-sales dataset total: **{branch_sales.iloc[0]['branch']}** ({branch_sales.iloc[0]['sales_amount']:,.2f}).")

# 2) Most profitable item
top_item = item_only.groupby('product_desc', as_index=False)['total_profit'].sum().sort_values('total_profit', ascending=False)
if not top_item.empty:
    insights.append(f"Most profitable item overall: **{top_item.iloc[0]['product_desc']}** ({top_item.iloc[0]['total_profit']:,.2f} profit).")

# 3) Largest loss item
loss_item = item_only.groupby('product_desc', as_index=False)['total_profit'].sum().sort_values('total_profit', ascending=True)
if not loss_item.empty:
    insights.append(f"Largest loss-making item overall: **{loss_item.iloc[0]['product_desc']}** ({loss_item.iloc[0]['total_profit']:,.2f} profit).")

# 4) Branch with highest profit margin in category totals
if not cat_branch_totals.empty:
    top_margin = cat_branch_totals.sort_values('profit_margin_pct', ascending=False).iloc[0]
    insights.append(f"Highest branch profit margin (category totals): **{top_margin['branch']}** ({top_margin['profit_margin_pct']:.2f}%).")

# 5) Data quality note from 00673 mismatch pattern
anomaly_count = 0
if {'item_branch_total_price', 'cat_branch_total_price'}.issubset(consistency.columns):
    anomaly_count = int((consistency['abs_diff'] > 1).sum())
insights.append(f"Cross-report check flagged **{anomaly_count}** branches with non-trivial total price differences between item-level and category-level totals.")

for i, text in enumerate(insights, 1):
    display(Markdown(f"{i}. {text}"))


## **Next Steps**

1. Validate branches with large cross-report differences against source exports.
2. Build a forecasting notebook from `rep_00134_comparative_monthly_sales_clean_long.csv`.
3. Add store clustering using branch-level KPIs (sales mix, margin, ticket size proxy).